In [11]:
from SPARQLWrapper import SPARQLWrapper, JSON
from googletrans import Translator
from transliterate import translit
import pandas as pd
import re
from collections import defaultdict
from time import sleep
import time
from tqdm import tqdm
from inltk.inltk import identify_language, reset_language_identifying_models,setup
import wptools
import json
import requests
from urllib.parse import  quote
import mwparserfromhell as mwp

In [13]:
class sparqlResults():
    
    def __init__(self,query_file):
        
        self.agent = 'Satakarni'
        
        with open(query_file, 'r') as file:
            self.query = file.read()
    
    def _val_cols(self):
        cols = list(self.df.columns.values)
        tmp = []
        for col in cols:
            if(col.split('.')[-1] == 'value'):
                tmp.append(col)
        return tmp
    
    def change_item(self,newItem):
        wikibase_item = newItem
        query_id = '?query_id ' + '{wd:' + wikibase_item + '}'
        self.query = re.sub(r'\?query_id \{.*\}', query_id, self.query)    
    
    def results(self):
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent=self.agent)
        sparql.setQuery(self.query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        self.df = pd.json_normalize(results['results']['bindings'])
        return self.df[self._val_cols()]

In [14]:
sparq_obj = sparqlResults('writer.rq')

In [15]:
results = sparq_obj.results()

In [16]:
results.head() # default Douglas Adams

,dateofbirth.value,dateofdeath.value,signature.value,placeofbirthLabel.value,birthnameLabel.value,placeofdeathLabel.value,restingplaceLabel.value,occupationLabel.value,nationalityLabel.value,educationLabel.value,genreLabel.value,notableworkLabel.value,awardLabel.value,spouseLabel.value,childrenLabel.value,websiteLabel.value
0,1952-03-11T00:00:00Z,2001-05-11T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कैम्ब्रिज,Douglas Noël Adams,सांता बारबरा,Highgate Cemetery,पटकथा लेखक,यूनाइटेड किंगडम,St John's College,विज्ञान कथा साहित्य,"Life, the Universe and Everything",Hugo Award for Best Dramatic Presentation,Jane Belson,Polly Adams,https://douglasadams.com/
1,1952-03-11T00:00:00Z,2001-05-11T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कैम्ब्रिज,Douglas Noël Adams,सांता बारबरा,Highgate Cemetery,लेखक,यूनाइटेड किंगडम,St John's College,विज्ञान कथा साहित्य,"Life, the Universe and Everything",Hugo Award for Best Dramatic Presentation,Jane Belson,Polly Adams,https://douglasadams.com/
2,1952-03-11T00:00:00Z,2001-05-11T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कैम्ब्रिज,Douglas Noël Adams,सांता बारबरा,Highgate Cemetery,नाटककार,यूनाइटेड किंगडम,St John's College,विज्ञान कथा साहित्य,"Life, the Universe and Everything",Hugo Award for Best Dramatic Presentation,Jane Belson,Polly Adams,https://douglasadams.com/
3,1952-03-11T00:00:00Z,2001-05-11T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कैम्ब्रिज,Douglas Noël Adams,सांता बारबरा,Highgate Cemetery,हास्य अभिनेता,यूनाइटेड किंगडम,St John's College,विज्ञान कथा साहित्य,"Life, the Universe and Everything",Hugo Award for Best Dramatic Presentation,Jane Belson,Polly Adams,https://douglasadams.com/
4,1952-03-11T00:00:00Z,2001-05-11T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कैम्ब्रिज,Douglas Noël Adams,सांता बारबरा,Highgate Cemetery,children's writer,यूनाइटेड किंगडम,St John's College,विज्ञान कथा साहित्य,"Life, the Universe and Everything",Hugo Award for Best Dramatic Presentation,Jane Belson,Polly Adams,https://douglasadams.com/


In [17]:
sparq_obj.change_item('Q7241')
results_2 = sparq_obj.results()

In [18]:
results_2.head() # Ranbindranath Tagore

,dateofbirth.value,dateofdeath.value,signature.value,placeofbirthLabel.value,placeofdeathLabel.value,occupationLabel.value,nationalityLabel.value,educationLabel.value,genreLabel.value,notableworkLabel.value,awardLabel.value,spouseLabel.value,childrenLabel.value
0,1861-05-07T00:00:00Z,1941-08-07T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कोलकाता,कोलकाता,कवि,ब्रिटिश राज,"यूनिवर्सिटी कॉलेज, लन्दन",उपन्यास,भानुसिंह ठाकुरेर पदावली,साहित्य में नोबेल पुरस्कार,Mrinalini Devi,Rathindranath Tagore
1,1861-05-07T00:00:00Z,1941-08-07T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कोलकाता,कोलकाता,गायक,ब्रिटिश राज,"यूनिवर्सिटी कॉलेज, लन्दन",उपन्यास,भानुसिंह ठाकुरेर पदावली,साहित्य में नोबेल पुरस्कार,Mrinalini Devi,Rathindranath Tagore
2,1861-05-07T00:00:00Z,1941-08-07T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कोलकाता,कोलकाता,नाटककार,ब्रिटिश राज,"यूनिवर्सिटी कॉलेज, लन्दन",उपन्यास,भानुसिंह ठाकुरेर पदावली,साहित्य में नोबेल पुरस्कार,Mrinalini Devi,Rathindranath Tagore
3,1861-05-07T00:00:00Z,1941-08-07T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कोलकाता,कोलकाता,कलाकार,ब्रिटिश राज,"यूनिवर्सिटी कॉलेज, लन्दन",उपन्यास,भानुसिंह ठाकुरेर पदावली,साहित्य में नोबेल पुरस्कार,Mrinalini Devi,Rathindranath Tagore
4,1861-05-07T00:00:00Z,1941-08-07T00:00:00Z,http://commons.wikimedia.org/wiki/Special:File...,कोलकाता,कोलकाता,गीतकार,ब्रिटिश राज,"यूनिवर्सिटी कॉलेज, लन्दन",उपन्यास,भानुसिंह ठाकुरेर पदावली,साहित्य में नोबेल पुरस्कार,Mrinalini Devi,Rathindranath Tagore


In [19]:
for col in results.columns.values:
    print(col,':',pd.unique(results[col]))
    

dateofbirth.value : ['1952-03-11T00:00:00Z']
dateofdeath.value : ['2001-05-11T00:00:00Z']
signature.value : ['http://commons.wikimedia.org/wiki/Special:FilePath/Douglas%20Adams%20signature.svg']
placeofbirthLabel.value : ['कैम्ब्रिज']
birthnameLabel.value : ['Douglas Noël Adams']
placeofdeathLabel.value : ['सांता बारबरा']
restingplaceLabel.value : ['Highgate Cemetery']
occupationLabel.value : ['पटकथा लेखक' 'लेखक' 'नाटककार' 'हास्य अभिनेता' "children's writer" 'उपन्यासकार' 'science fiction writer']
nationalityLabel.value : ['यूनाइटेड किंगडम']
educationLabel.value : ["St John's College" 'Brentwood School']
genreLabel.value : ['विज्ञान कथा साहित्य' 'प्रहसन' 'कटूपहास']
notableworkLabel.value : ['Life, the Universe and Everything' "The Hitchhiker's Guide to the Galaxy pentalogy" 'Mostly Harmless'
 'The Restaurant at the End of the Universe' 'So Long, and Thanks for All the Fish' 'The Private Life of Genghis Khan'
 'Dirk Gently tetralogy']
awardLabel.value : ['Hugo Award for Best Dramatic Pre

In [20]:
import pickle
with open('parsed_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [11]:
instance_obj = sparqlResults('instanceOf.rq')
instances = defaultdict(list)
errors = []
values = data.values()
start = time.time()
i = 1
count = 0
for doc in tqdm(values):
#     sleep(0.3)
    instance_obj = sparqlResults('instanceOf.rq')
    instance_obj.change_item(doc['wikibase_item'])
    try:
        df = instance_obj.results()
        if(not df.empty):
            instances[df['instanceLabel.value'].iloc[0]].append({'item' : doc['wikibase_item'], 'infobox' : doc['infobox']})
        else:
            instances['None'].append({'item' : doc['wikibase_item'], 'infobox' : doc['infobox']})
    except:
        errors.append(doc['wikibase_item'])
    if(i%100 == 0):
        with open('./stats/instance_' + str(count) + '.pkl','wb') as handle:
            pickle.dump(instances, handle)
        instances = defaultdict(list)
        count += 1
    i += 1
end = time.time()
print(end-start)

100%|██████████| 145805/145805 [00:00<00:00, 1382419.28it/s]

0.1123189926147461
113031


In [67]:
# -*- coding: utf-8 -*-
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
# print(isEnglish('कोलकाता'))

def Find_URL(string): 
    # findall() has been used  
    # with valid conditions for urls in string 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    if "URL" in string:
        check = True
    else:
        check = False
    url = re.findall(regex,string)  
    if url or check:
        return True
    else:
        return False

def indic_translate(eng):
    hi = []
    for i in eng:
        text=quote(i)
        # text=quote('मनिश् जोए')
        url='http://www.cfilt.iitb.ac.in/indicnlpweb/indicnlpws/translate/en/hi/{}/'.format(text)
        ## Note the forward slash '/' at the end of the URL. It's should be there, but please live with it for now!

        # print(url)
        response = requests.get(url)
        hi.append(response.json()['hi'])
    return hi

proxies=['translate.google.com','translate.google.co.kr']
translator = Translator(service_urls=proxies)
count = 0
total = 0
google = 0
for col in results.columns.values:
    hindi = []
    for vals in pd.unique(results[col]):
        total += 1
        if isEnglish(vals) and not Find_URL(vals):
            indic_translated = indic_translate([vals])
            check = True
            for i in indic_translated[0].split(' '):
                if isEnglish(i):
                    check = False
            if check:
                hindi = hindi + indic_translated
            else:
                google +=1
                translations = translator.translate([vals], dest = "hi")
                for translation in translations:
                    hindi.append(translation.text)
        else:
            if not Find_URL(vals):
                try:
                    vals = translit(vals,reversed = True)
                    indic_translated = indic_translate([vals])
                    check = True
                    for i in indic_translated[0].split(' '):
                        if isEnglish(i):
                            check = False
                    if check:
                        hindi = hindi + indic_translated
                    else:
                        google +=1
                        translations = translator.translate([vals], dest = "hi")
                        for translation in translations:
                            hindi.append(translation.text)
                except:
    #                 hindi = hindi + [vals]
                    hindi.append(vals)
            else:
                hindi.append(vals)
    print(col,':',hindi)
    count+=1
#     if count == 5:
#         break
print(total,google)

dateofbirth.value : ['1952-03-11T00: 00: 00Z']
dateofdeath.value : ['2001-05-11T00: 00: 00Z']
signature.value : ['http://commons.wikimedia.org/wiki/Special:FilePath/Douglas%20Adams%20signature.svg']
placeofbirthLabel.value : ['कैम्ब्रिज']
birthnameLabel.value : ['डगलस नोएल एडम्स']
placeofdeathLabel.value : ['सांता बारबरा']
restingplaceLabel.value : ['हाईगेट कब्रिस्तान']
occupationLabel.value : ['पटकथा लेखक', 'लेखक', 'नाटककार', 'हास्य अभिनेता', 'बच्चों के लेखक', 'उपन्यासकार', 'विज्ञान कथा लेखक']
nationalityLabel.value : ['यूनाइटेड किंगडम']
educationLabel.value : ['सेंट जॉन्स कॉलेज', 'ब्रेंटवुड स्कूल']
genreLabel.value : ['विज्ञान कथा साहित्य', 'प्रहसन', 'कटूपहास']
notableworkLabel.value : ['जीवन, ब्रह्मांड और सब कुछ', 'द हिचहाइकर गाइड टू द गैलेक्सी पैथोलॉजी', 'ज्यादातर हानिरहित', 'ब्रह्मांड के अंत में रेस्तरां', 'इतने लंबे समय और सारी मछलियों के लिए धन्यवाद', 'चंगेज खान की निजी जिंदगी', 'डिर्क जेंटली टेट्रालॉजी']
awardLabel.value : ['सर्वश्रेष्ठ नाटकीय प्रस्तुति के लिए ह्यूगो पुरस्कार',

In [63]:
entity_id = 'Q42'
titles = wptools.page(wikibase = entity_id).get_wikidata()
print(titles.data['title'])
title_list = titles.data['title'].split('_')
title = " ".join(title_list)
page = wptools.page(title)
page.get_parse()

www.wikidata.org (wikidata) Q42
www.wikidata.org (labels) P69|P463|P2373|Q2576795|P1728|P5340|P72...
www.wikidata.org (labels) P2581|Q36180|Q463035|P106|P3222|P2734|Q...
www.wikidata.org (labels) P535|P1953|P3192|Q9531|Q6625963|P271|Q2...
www.wikidata.org (labels) P6262|P1207|P947|P268|P19|P244|P2191|P8...
www.wikidata.org (labels) P1280|P2638|P213|Q159288|P2163|P1266|Q9...
www.wikidata.org (labels) P2469|P136|P3630|P349|P6620|P735|Q35206...
en.wikipedia.org (imageinfo) File:Douglas adams portrait cropped.jpg
Douglas Adams (en) data
{
  aliases: <list(3)> Douglas Noel Adams, Douglas Noël Adams, Dougl...
  claims: <dict(212)> P31, P21, P106, P800, P569, P19, P570, P1196...
  description: English writer and humorist
  image: <list(1)> {'file': 'File:Douglas adams portrait cropped.j...
  label: Douglas Adams
  labels: <dict(263)> P69, P463, P2373, Q2576795, P1728, P5340, P7...
  modified: <dict(1)> wikidata
  requests: <list(8)> wikidata, labels, labels, labels, labels, la...
  title: Dou

Douglas_Adams


en.wikipedia.org (imageinfo) File:Douglas adams portrait cropped.jpg
Douglas Adams (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Douglas a...
  infobox: <dict(15)> name, image, birth_name, birth_date, birth_p...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 8091
  parsetree: <str(75787)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Douglas Adams
  wikibase: Q42
  wikidata_url: https://www.wikidata.org/wiki/Q42
  wikitext: <str(62492)> {{short description|English author and hu...
}


In [70]:
key_en = list(page.data['infobox'].keys())
value_en = list(page.data['infobox'].values())
infobox = []
def is_date(text):
    if ('{birth' in text.lower()) or ('{death' in text.lower()):
        return True
    return False
    
count =0

for ind in range(len(key_en)):
    if('flatlist' in value_en[ind]):
        value_en[ind] = value_en[ind].strip('{}').strip('flatlist|').strip().replace('\n',', ')
#         print(value_en[i])    
    if not Find_URL(value_en[ind]) and not is_date(value_en[ind]):
        vals = mwp.parse(value_en[ind]).strip_code().replace('*','')
    else:
        vals = value_en[ind]
#     print(vals)
    hindi = []
    if not is_date(vals):
        if isEnglish(vals) and not Find_URL(vals):
            indic_translated = indic_translate([vals])
            check = True
            for i in indic_translated[0].split(' '):
                if isEnglish(i):
                    check = False
            if check:
                hindi = hindi + indic_translated
            else:
                google +=1
                translations = translator.translate([vals], dest = "hi")
                for translation in translations:
                    hindi.append(translation.text)
        else:
            if not Find_URL(vals):
                try:
                    vals = translit(vals,reversed = True)
                    indic_translated = indic_translate([vals])
                    check = True
                    for i in indic_translated[0].split(' '):
                        if isEnglish(i):
                            check = False
                    if check:
                        hindi = hindi + indic_translated
                    else:
                        google +=1
                        translations = translator.translate([vals], dest = "hi")
                        for translation in translations:
                            hindi.append(translation.text)
                except:
    #                 hindi = hindi + [vals]
                    hindi.append(vals)
            else:
                hindi.append(vals)
    else:
        hindi.append(vals)
#     print(hindi[0])
#     print(key_en[ind])
    infobox.append([key_en[ind]+" = "+hindi[0]])
infobox

[['name = डगलस एडम्स'],
 ['image = डगलस एडम्स पोट्रेट क्रॉप्ड.जेपीजी'],
 ['birth_name = डगलस नोएल एडम्स'],
 ['birth_date = {{birth date|1952|3|11|df|=|yes}}'],
 ['birth_place = कैम्ब्रिज, कैम्ब्रिजशायर, इंग्लैंड'],
 ['death_date = {{Death date and age|2001|5|11|1952|3|11|df|=|yes}}'],
 ['death_place = मॉन्टेसिटो, कैलिफ़ोर्निया, यू.एस.'],
 ['resting_place = हाईगेट कब्रिस्तान, लंदन, इंग्लैंड'],
 ['occupation = लेखक'],
 ['alma_mater = सेंट जॉन्स कॉलेज, कैम्ब्रिज'],
 ['genre = साइंस फिक्शन, कॉमेडी, व्यंग्य'],
 ['notablework = आकाशगंगा के लिए Hitchhiker गाइड'],
 ['signature = डगलस एडम्स का हस्ताक्षर (क्रॉप्ड) .jpg'],
 ['awards = इंकपॉट अवार्ड (1983)'],
 ['website = {{URL|douglasadams.com}}']]